In [1]:
import numpy as np
import pandas as pd
import os
#from osgeo import gdal
import geopandas as gpd
#import rasterio
import osmnx as ox
import rioxarray
import pyproj
import shapely.geometry as sg
import networkx as nx
import momepy

In [2]:
# Test data
test_path = "C:/Users/ygrin/Documents/Studie - MSc ADS/Utrecht University/Block 4 - Thesis/TestData/"
single_point = gpd.read_file(test_path + "Test_single_home_location.gpkg")
multi_point = gpd.read_file(test_path + "Test_multiple_home_locations.gpkg")
#polygon = gpd.read_file(test_path + "TestArea.gpkg")
ndvi = rioxarray.open_rasterio(test_path + "NDVI_data_test.tif")
land_cover = rioxarray.open_rasterio(test_path + "Landcover_data_test.tif")
#network = gpd.read_file(test_path + "test_network.gpkg", layer='edges')
network = gpd.read_file(test_path + "test_network_shp.shp")
polygon = gpd.read_file(test_path + "test_polygon.gpkg")
canopy_single = gpd.read_file(test_path + "Canopy_single_home_500m.gpkg")

single_point_geographic = single_point.to_crs('epsg:4326')

# Greenness Accessibility -- Shortest Network Distance to Park

In [3]:
def get_shortest_distance_park(point_of_interest_file, buffer_dist=500, network_type='walk', park_vector_file=None, network_file=None, 
                               output_dir=os.getcwd()):
    # Read and process user input, check conditions
    poi = gpd.read_file(point_of_interest_file)
    if all(poi['geometry'].geom_type == 'Point') or all(poi['geometry'].geom_type == 'Polygon'):
        geom_type = poi.iloc[0]['geometry'].geom_type
    else:
        raise ValueError("Please make sure all geometries are of 'Point' type or all geometries are of 'Polygon' type and re-run the function")

    if not poi.crs.is_projected:
        raise ValueError("The CRS of the PoI dataset is currently geographic, please transform it into a local projected CRS and re-run the function")
    else:
        epsg = poi.crs.to_epsg()

    # In case of house polygons, transform to centroids
    if geom_type == "Polygon":
        print("Changing geometry type to Point by computing polygon centroids so that network distance can be retrieved...")
        poi['geometry'] = poi['geometry'].centroid
        print("Done \n")

    if "id" in poi.columns:
        if poi['id'].isnull().values.any():
            poi['id'] = poi['id'].fillna(pd.Series(range(1, len(poi) + 1))).astype(int)
    else:
        poi['id'] = pd.Series(range(1, len(poi) + 1)).astype(int)

    if not isinstance(buffer_dist, int) or (not buffer_dist > 0):
        raise TypeError("Please make sure that the buffer distance is set as a positive integer")

    if network_type not in ["walk", "bike", "drive", "all"]:
        raise ValueError("Please make sure that the network_type argument is set to either 'walk', 'bike, 'drive' or 'all', and re-run the function")

    epsg_transformer = pyproj.Transformer.from_crs(f"epsg:{epsg}", "epsg:4326") # EPSG transformer to use for OSM            
    # Read park polygons, retrieve from OSM if not provided by user 
    if park_vector_file is not None:
        park_src = gpd.read_file(park_vector_file)
        if not park_src.crs.to_epsg() == epsg:
            print("Adjusting CRS of Park file to match with Point of Interest CRS...")
            park_src.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")

        print("Filtering parks which are (partly) within buffer zone...")
        #################################################################
        print{"Done \n"}
    else:
        print("Retrieving parks within buffer distance for point(s) of interest...")
        park_tags = {'leisure': 'park', 'boundary': 'national_park', 'landuse': 'recreation_ground'}
        parks_buffer = gpd.GeoDataFrame()
        for geom in multi_point['geometry']:
            latlon = epsg_transformer.transform(geom.x, geom.y)
            park_geom = ox.geometries_from_point(latlon, tags=park_tags, dist=buffer_dist)
            parks_buffer = gpd.GeoDataFrame(pd.concat([parks_buffer, park_geom], ignore_index=True), crs=park_geom.crs)
        parks_buffer.to_crs(f"EPSG:{epsg}", inplace=True)
        print("Done \n")

    # Read road network, retrieve from OSM if not provided by user 
    if network_file is not None:
        network = gpd.read_file(network_file)
        if not network.crs.to_epsg() == epsg:
            print("Adjusting CRS of Network file to match with Point of Interest CRS...")
            network.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")
    else:
        print("Retrieving infrastructure network within buffer distance for point(s) of interest...")
        network = gpd.GeoDataFrame()
        for geom in multi_point['geometry']:
            latlon = epsg_transformer.transform(geom.x, geom.y)
            park_geom = ox.geometries_from_point(latlon, tags=park_tags, dist=500)
            park_src = gpd.GeoDataFrame(pd.concat([park_src, park_geom], ignore_index=True), crs=park_geom.crs)
        print("Done \n")
        

    print("Writing results to new geopackage file in specified directory...")
    input_filename, ext = os.path.splitext(point_of_interest_file)
    poi.to_file(os.path.join(output_dir, f"{input_filename}_ShortDistPark_added.gpkg"), driver="GPKG")
    print("Done")

    return poi

In [43]:
epsg_transformer = pyproj.Transformer.from_crs("epsg:27700", "epsg:4326") # EPSG transformer to use for OSM
park_tags = {'leisure': 'park', 'boundary': 'national_park', 'landuse': 'recreation_ground'}
park_src = gpd.GeoDataFrame()
for geom in multi_point['geometry']:
    latlon = epsg_transformer.transform(geom.x, geom.y)
    park_geom = ox.geometries_from_point(latlon, tags=park_tags, dist=500)
    park_src = gpd.GeoDataFrame(pd.concat([park_src, park_geom], ignore_index=True), crs=park_geom.crs)
park_src.to_crs("EPSG:27700", inplace=True)

In [53]:
epsg_transformer = pyproj.Transformer.from_crs("epsg:27700", "epsg:4326") # EPSG transformer to use for OSM
park_tags = {'leisure': 'park', 'boundary': 'national_park', 'landuse': 'recreation_ground'}
park_src = []
for geom in multi_point['geometry']:
    latlon = epsg_transformer.transform(geom.x, geom.y)
    park_geom = ox.geometries_from_point(latlon, tags=park_tags, dist=500)
    parks = gpd.GeoDataFrame(park_geom, crs=park_geom.crs).to_crs("EPSG:27700")
    park_src.append(parks)

In [61]:
# define the EPSG transformer for transforming the point geometries
epsg_transformer = pyproj.Transformer.from_crs("epsg:27700", "epsg:4326")
# create an empty GeoDataFrame to hold the network geometries
network_list = []
# iterate through each point geometry in the multi_point GeoDataFrame
for geom in multi_point['geometry']:
    # transform the point geometry to lat/lon coordinates
    latlon = epsg_transformer.transform(geom.x, geom.y)
    # retrieve the network geometries within the buffer distance from the point
    network_graph = ox.graph_from_point(latlon, dist=500, network_type='walk')
    network_graph = ox.project_graph(network_graph, to_crs="EPSG:27700")
    network_list.append(network_graph)

# create an empty MultiDiGraph object to hold the combined graphs
#combined_graph = nx.MultiDiGraph()

# iterate through each graph in the list and add its nodes and edges to the combined graph
#for graph in network_list:
#    combined_graph = nx.union(combined_graph, graph)

#combined_graph = ox.project_graph(combined_graph, to_crs="EPSG:27700")


In [ ]:
for geom, parks, graph in zip(multi_point['geometry'], park_src, network_list):
    nx.shortest_path_length(graph, source=geom, target=)